# Model experiments for Animals classification

In [26]:
from enum import Enum
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import pandas as pd
from typing import Any
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Dataset

In [13]:
class EnglishLabel(str, Enum):
    DOG = "dog"
    HORSE = "horse"
    ELEPHANT = "elephant"
    BUTTERFLY = "butterfly"
    CHICKEN = "chicken"
    CAT = "cat"
    COW = "cow"
    SHEEP = "sheep"
    SPIDER = "spider"
    SQUIRREL = "squirrel"


class ItalianLabel(str, Enum):
    CANE = "cane"
    CAVALLO = "cavallo"
    ELEFANTE = "elefante"
    FARFALLA = "farfalla"
    GALLINA = "gallina"
    GATTO = "gatto"
    MUCCA = "mucca"
    PECORA = "pecora"
    RAGNO = "ragno"
    SCOIATTOLO = "scoiattolo"


def translate_labels(labels: list[EnglishLabel]) -> list[ItalianLabel]:
    translate = {
        EnglishLabel.DOG: ItalianLabel.CANE,
        EnglishLabel.HORSE: ItalianLabel.CAVALLO,
        EnglishLabel.ELEPHANT: ItalianLabel.ELEFANTE,
        EnglishLabel.BUTTERFLY: ItalianLabel.FARFALLA,
        EnglishLabel.CHICKEN: ItalianLabel.GALLINA,
        EnglishLabel.CAT: ItalianLabel.GATTO,
        EnglishLabel.COW: ItalianLabel.MUCCA,
        EnglishLabel.SHEEP: ItalianLabel.PECORA,
        EnglishLabel.SPIDER: ItalianLabel.RAGNO,
        EnglishLabel.SQUIRREL: ItalianLabel.SCOIATTOLO,
    }
    return [translate[label] for label in labels]


def build_dataframes(
    data_dir: str | Path, english_labels: list[EnglishLabel], test_size: float = 0.2
) -> tuple[pd.DataFrame, pd.DataFrame, int]:
    data_dir = Path(data_dir)
    if not data_dir.exists() or not data_dir.is_dir():
        raise ValueError(f"Invalid data directory: {data_dir}")

    italian_labels = translate_labels(english_labels)
    num_labels = len(italian_labels)

    data = []
    for label_idx, italian_label in enumerate(italian_labels):
        label_dir = data_dir / italian_label.value
        for fpath in label_dir.glob("*"):
            if fpath.is_file():
                data.append(
                    {
                        "italian_label": italian_label,
                        "label_idx": label_idx,
                        "path": str(fpath),
                    }
                )
    data_df = pd.DataFrame(data)

    train_df, test_df = train_test_split(data_df, test_size=test_size)
    return train_df, test_df, num_labels


class AnimalsDataset(Dataset):
    def __init__(
        self, data_df: pd.DataFrame, num_labels: int, transform: Any | None = None
    ):
        self.data_df = data_df
        self.num_labels = num_labels
        self.transform = transform

    def __len__(self) -> int:
        return len(self.data_df)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        entry = self.data_df.iloc[idx]
        image = read_image(entry["path"])
        if self.transform is not None:
            image = self.transform(image)

        labels = np.zeros(self.num_labels, dtype=int)
        labels[entry["label_idx"]] = 1
        return image, torch.tensor(labels)

In [ ]:
train_df, test_df, num_labels = build_dataframes(
    "../data/raw-img", [EnglishLabel.DOG, EnglishLabel.HORSE], test_size=0.2
)

### ResNet-18

In [19]:
from torchvision.models import resnet18, ResNet18_Weights

In [ ]:
train_dataset = AnimalsDataset(
    train_df, num_labels, ResNet18_Weights.DEFAULT.transforms()
)
test_dataset = AnimalsDataset(
    test_df, num_labels, ResNet18_Weights.DEFAULT.transforms()
)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [34]:
model = resnet18(weights=ResNet18_Weights.DEFAULT)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\makarevi/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100.0%


In [40]:
num_epochs = 3

running_loss = 0.0

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels.argmax(axis=1))
        loss.backward()
        optimizer.step()

        print(f"{loss.item() = }")

        running_loss += loss.item()
        if i % 10 == 9:
            avg_loss = running_loss / 10
            print(f"{i = }. {avg_loss = }")
            running_loss = 0.0

loss.item() = 11.885480880737305
loss.item() = 5.318265438079834
loss.item() = 2.412975549697876
loss.item() = 1.8450558185577393


KeyboardInterrupt: 